In [2]:
import cv2
from PIL import Image
import requests
from bs4 import BeautifulSoup
import base64
import os

In [3]:
emojiFolder = "Emojis"

# Get the web page containing the list of emojis
url = "https://unicode.org/emoji/charts/full-emoji-list.html"
response = requests.get(url)

# Parse the web page using BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")


In [4]:
rows = soup.find_all('tr')

In [5]:
def getAverageColor(img):
    img = img.convert('RGBA')
    pixels = img.getdata()

    total = 0 
    r, g, b = 0, 0, 0
    for pixel in pixels:
        r += pixel[0]
        g += pixel[1]
        b += pixel[2]
        total += 1

    return (r/total, g/total, b/total)



In [22]:
emojis = {}
for row in rows:
    emoji_ID = row.find('td', class_='rchars')
    if emoji_ID is None:
        continue
    emoji_ID = emoji_ID.text

    #Get The Emoji
    emoji= row.find('td', class_='chars')
    if emoji is None:
        continue
    emoji = emoji.text

    #Filter out the emojis that are not supported by the font
    if emoji == "🏻" or emoji == "🏼" or emoji == "🏽" or emoji == "🏾" or emoji == "🏿" or emoji == "🫏":
        continue

    img_url = row.find('img')['src']
    img_data = img_url.split(',')[1]

    img_data = base64.b64decode(img_data)

    img_path = emojiFolder + "/" + emoji_ID + ".png"
    with open(img_path, "wb") as fh:
        fh.write(img_data)

    #Get The Average Color of the emoji
    img = Image.open(img_path)
    avgColor = getAverageColor(img)
    #Convert to int
    avgColor = (int(avgColor[0]), int(avgColor[1]), int(avgColor[2]))

    



    #Add the emoji to the dictionary
    emojis[emoji_ID] = {
        "image": cv2.imread(emojiFolder + "/" + emoji_ID + ".png"),
        "color": avgColor,
        "emoji": emoji
    }






In [23]:
for emoji in emojis:
    #Create an image with the average color of the emoji
    img = Image.new('RGB', (10, 10), color = emojis[emoji]["color"])
    img.save(emojiFolder + "/" + emoji + "_color.png")


In [24]:
#Using a color (from image.getpixel) return the emoji color that is closest to it

def getDistance(color1, color2):
    # Get the distance between the two colors
    distance = ((color1[0] - color2[0]) ** 2 + (color1[1] - color2[1]) ** 2 + (color1[2] - color2[2]) ** 2) ** 0.5
    return distance

def getClosestEmoji(color):
    # Get the distance between the color and each emoji color
    distances = {}
    for emoji in emojis:
        distances[emoji] = getDistance(color, emojis[emoji]["color"])
    
    # Get the emoji with the shortest distance
    closestEmoji = min(distances, key=distances.get)
    
    return emojis[closestEmoji]["emoji"]


In [26]:
image = Image.open('/Users/mattraghu/Downloads/ATENA.png')

width = 77
# Resize the image to match the width, maintain aspect ratio
image = image.resize((width, int(image.height * width / image.width)))

text = ""
for y in range(image.height):
    for x in range(image.width):
        # Get the color of the pixel
        color = image.getpixel((x, y))
        
        # Get the closest emoji
        emoji = getClosestEmoji(color)

        if emoji == "🫏":
            print("Yes")
        
        # Add the emoji to the text
        text += emoji

    # Add a new line
    text += "\n"

#Write to file
with open("output.txt", "w") as f:
    f.write(text)

